### 문제 1-1 기본 Chain 만들기 AI- 요리사 

In [33]:
from dotenv import load_dotenv
import os
# .env 파일을 불러와서 환경 변수로 설정
load_dotenv(dotenv_path='../.env')

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
print(OPENAI_API_KEY[:5])

gsk_w


In [34]:
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

# 프롬프트 템플릿 (출력 형식 예시 포함)
template_text = """
당신은 요리사입니다.
주어진 재료({ingredients})로 만들 수 있는 대표 요리 하나를 추천하고,
반드시 아래 출력 형식을 따라주세요.

출력 형식:
추천 요리: (요리명)
재료: (사용된 재료 나열)
조리법:
1) ...
2) ...
3) ...
"""

prompt_template = PromptTemplate.from_template(template_text)

# LLM 초기화
llm = ChatOpenAI(
    api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",
    model="openai/gpt-oss-120b",
    temperature=0.7
)

# 체인 생성
chain = prompt_template | llm | StrOutputParser()


In [35]:
# 실행
response = chain.invoke({"ingredients": "계란, 밥, 김치"})
print(response)

추천 요리: 김치볶음밥  
재료: 밥, 김치, 계란  
조리법:  
1) 팬에 기름을 두르고 잘게 썬 김치를 넣어 중불에서 2~3분 정도 볶아 향을 낸다.  
2) 밥을 넣고 김치와 함께 고루 섞으며 볶아, 밥이 따뜻해지고 김치와 잘 어우러지도록 5분 정도 볶는다.  
3) 마지막으로 풀어 놓은 계란을 팬 가장자리에 붓고 스크램블하듯 저어가며 전체와 섞어 익히면 완성이다.  


### 문제 1-2 Multi Chain 만들기 - 여행지 정보 시스템

In [36]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableLambda

# 1단계: 명소 추천
stage1_prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 세계 여행 전문가입니다. 사용자가 말한 도시/국가에서 대표 명소 1가지를 추천하세요. 다른 설명은 하지 말고 명소 이름만 출력하세요."),
    ("user", "여행지: {destination}")
])
stage1_chain = stage1_prompt | llm | StrOutputParser()

# 2단계: 상세 정보
stage2_prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 여행지 설명 전문가입니다. 주어진 명소에 대해 간단하게 역사, 특징, 방문 팁을 한국어로 각각 2줄 이내로 정리하세요."),
    ("user",
     "명소: {place}\n\n"
     "출력 형식:\n"
     "명소: {place}\n"
     "역사:\n- ...\n- ...\n"
     "특징:\n- ...\n- ...\n"
     "방문 팁:\n- ...\n- ...")
])
stage2_chain = stage2_prompt | llm | StrOutputParser()

# 실행 예시
destination = "뉴욕"
landmark = stage1_chain.invoke({"destination": destination}).strip()
print("1단계 결과:", landmark)

details = stage2_chain.invoke({"place": landmark})
print("\n추천 2단계 결과:\n" + details)


1단계 결과: 자유의 여신상

추천 2단계 결과:
명소: 자유의 여신상  
역사:  
- 1886년 프랑스가 미국 독립 100주년을 기념해 선물한 동상.  
- 건축가 프레드릭 오귀스트 바르톨디와 조각가 아돌프 바르톨디가 설계.  

특징:  
- 46 m(기초 포함 93 m)의 거대한 구리 조각상으로, 횃불은 세계 평화와 자유의 상징.  
- 입장 시 왕관 내부까지 올라가 뉴욕 항구와 맨해튼 전경을 조망 가능.  

방문 팁:  
- 입장권은 사전 온라인 예약이 필수이며, 왕관 티켓은 조기 매진.  
- 보안 검사를 대비해 가벼운 복장과 큰 가방은 피하고, 일찍 방문하면 대기 시간을 줄일 수 있다.


### 문제 1-3 FewShotPromptTemplate과 시스템 메시지 활용 

In [45]:
from langchain_core.prompts import (
    FewShotChatMessagePromptTemplate, 
    ChatPromptTemplate
)
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
import os
from dotenv import load_dotenv

# .env 파일을 불러와서 환경 변수로 설정
load_dotenv(dotenv_path='../.env')

# Groq API 키를 환경 변수에서 가져옵니다.
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

# 1. 예시 데이터 (examples)
# 다양한 분야의 뉴스 예시를 제공하여 모델이 일관된 패턴을 학습하도록 돕습니다.
examples = [
    {
        "news": "삼성전자가 내년 초에 자체적으로 개발한 인공지능(AI) 가속기를 처음으로 출시할 예정이다. 이는 AI 반도체 시장에서 지배적인 위치를 차지하고 있는 엔비디아의 독점을 도전하고, 세계 최고의 반도체 제조업체로서의 지위를 다시 확립하려는 삼성전자의 노력으로 해석된다.",
        "keywords": "삼성전자, 인공지능, 엔비디아"
    },
    {
        "news": "세계보건기구(WHO)는 최근 새로운 건강 위기에 대응하기 위해 국제 협력의 중요성을 강조했다. 전염병 대응 역량의 강화와 글로벌 보건 시스템의 개선이 필요하다고 발표했다.",
        "keywords": "세계보건기구, 건강위기, 국제협력"
    },
    {
        "news": "테슬라의 일론 머스크 최고경영자(CEO)가 최근 주주총회에서 완전 자율주행(FSD) 기술의 발전 상황을 발표했다. 머스크는 FSD가 곧 인간 운전자를 능가할 것이라고 자신감을 내비쳤다.",
        "keywords": "테슬라, 일론 머스크, 자율주행"
    },
    {
        "news": "2024년 파리 올림픽이 7월 26일 개막식을 시작으로 17일간의 열전에 돌입했다. 전 세계 선수들이 각국을 대표해 총 32개 종목에서 메달을 놓고 경쟁을 펼친다.",
        "keywords": "파리 올림픽, 개막식, 메달"
    }
]

# 2. 예시 프롬프트 (example_prompt)
# 대화형 프롬프트 형식을 정의합니다.
example_prompt = ChatPromptTemplate.from_messages([
    ("human", "{news}"),
    ("ai", "키워드: {keywords}")
])

# 3. Few-Shot 프롬프트
# 위에서 정의한 예시들을 Few-Shot 프롬프트 템플릿에 담습니다.
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples
)

# 4. 최종 프롬프트 (final_prompt)
# 시스템 메시지, 예시 프롬프트, 실제 입력값 순서로 최종 프롬프트를 구성합니다.
final_prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 뉴스 기사에서 핵심 키워드 3개를 추출하는 전문가입니다. 뉴스 텍스트를 분석하여 가장 중요한 키워드 3개를 쉼표로 구분하여 출력하세요. 출력 형식은 '키워드: 키워드1, 키워드2, 키워드3'을 엄격하게 지켜야 합니다."),
    few_shot_prompt,
    ("human", "{input}")
])

# 5. LLM 및 체인 설정
# Groq API에서 현재 지원하는 모델명을 사용합니다.
llm = ChatOpenAI(
    api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",
    model="mixtral-8x7b-32768",  # 최신 Groq 모델로 변경
    temperature=0.1
)

chain = final_prompt | llm | StrOutputParser()

# 6. 테스트 뉴스
test_news = "제미나이 2.0 플래시는 현재 구글 AI 스튜디오(Google AI Studio) 및 버텍스 AI(Vertex AI)에서 제미나이 API를 통해 개발자에게 실험 모델로 제공됩니다. 모든 개발자는 멀티모달 입력 및 텍스트 출력을 사용할 수 있으며, 텍스트 음성 변환(text-to-speech) 및 네이티브 이미지 생성은 일부 파트너들을 대상으로 제공됩니다. 내년 1월에는 더 많은 모델 사이즈와 함께 일반에 공개될 예정입니다."

# 체인 실행
response = chain.invoke({"input": test_news})

# 결과 출력
print(response)

BadRequestError: Error code: 400 - {'error': {'message': 'The model `mixtral-8x7b-32768` has been decommissioned and is no longer supported. Please refer to https://console.groq.com/docs/deprecations for a recommendation on which model to use instead.', 'type': 'invalid_request_error', 'code': 'model_decommissioned'}}